In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Initialize Spark Session with Kafka package
spark = SparkSession.builder \
    .appName("KafkaSparkStreaming") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .config("spark.sql.shuffle.partitions", "2") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4e5e5a72-1491-4d8e-b03f-536c12c39fae;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-

In [2]:
# Kafka configuration
kafka_bootstrap_servers = "kafka:9092"  # Using container name since they're on same network
topic = "AIDHOOSTATION"

# Define schema for your data (adjust based on your message structure)
schema = StructType([
    StructField("key", StringType(), True),
    StructField("value", StringType(), True),
    # Add more fields as per your data structure
])

# Read from Kafka
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", topic) \
    .option("startingOffsets", "earliest") \
    .load()

In [3]:
# Cast the key and value from binary to string
kafka_df = df.select(
    col("key").cast("string"),
    col("value").cast("string"),
    col("timestamp"),
    col("topic"),
    col("partition"),
    col("offset")
)

In [4]:
# Process the data (example: simple printing)
query = kafka_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime="10 seconds") \
    .start()

# Keep the streaming running
query.awaitTermination()

25/03/15 08:27:55 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b4ec9948-2938-4e18-8edf-05f880c8c02b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/03/15 08:27:55 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/03/15 08:27:56 ERROR MicroBatchExecution: Query [id = 8fa3b12e-d5ac-4d4b-b885-2c9324d0b434, runId = c1add1f4-9328-4062-844f-5e12d3f384e6] terminated with error
java.lang.NoClassDefFoundError: org/apache/spark/kafka010/KafkaConfigUpdater
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$.kafkaParamsForDriver(KafkaSourceProvider.scala:645)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaScan.toMicroBatchStream(KafkaSourceProvider.scala:482)
	at org.ap

StreamingQueryException: org/apache/spark/kafka010/KafkaConfigUpdater
=== Streaming Query ===
Identifier: [id = 8fa3b12e-d5ac-4d4b-b885-2c9324d0b434, runId = c1add1f4-9328-4062-844f-5e12d3f384e6]
Current Committed Offsets: {}
Current Available Offsets: {}

Current State: INITIALIZING
Thread State: RUNNABLE

In [5]:
import os
print(os.listdir('/opt/spark/jars/'))

['hive-shims-scheduler-2.3.9.jar', 'hk2-api-2.6.1.jar', 'jakarta.validation-api-2.0.2.jar', 'commons-logging-1.1.3.jar', 'audience-annotations-0.5.0.jar', 'logging-interceptor-3.12.12.jar', 'antlr4-runtime-4.8.jar', 'spark-hive_2.12-3.2.1.jar', 'jaxb-api-2.2.11.jar', 'scala-library-2.12.15.jar', 'jdom-1.1.jar', 'hadoop-cloud-storage-3.3.1.jar', 'spark-mesos_2.12-3.2.1.jar', 'azure-data-lake-store-sdk-2.3.9.jar', 'breeze-macros_2.12-1.2.jar', 'breeze_2.12-1.2.jar', 'aopalliance-repackaged-2.6.1.jar', 'commons-collections-3.2.2.jar', 'kubernetes-model-discovery-5.4.1.jar', 'spire-macros_2.12-0.17.0.jar', 'bonecp-0.8.0.RELEASE.jar', 'dropwizard-metrics-hadoop-metrics2-reporter-0.1.2.jar', 'spark-unsafe_2.12-3.2.1.jar', 'commons-text-1.6.jar', 'spark-graphx_2.12-3.2.1.jar', 'arpack-2.2.1.jar', 'commons-crypto-1.1.0.jar', 'zjsonpatch-0.3.0.jar', 'pyrolite-4.30.jar', 'metrics-json-4.2.0.jar', 'zstd-jni-1.5.0-4.jar', 'jersey-container-servlet-2.34.jar', 'json4s-core_2.12-3.7.0-M11.jar', 'hk2-

In [9]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
    .appName("KafkaSparkStreaming") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar") \
    .config("spark.driver.extraClassPath", "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar") \
    .config("spark.executor.extraClassPath", "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

# Define Kafka DataFrame (adjust topic and bootstrap servers as needed)
kafka_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "AIDHOOSTATION") \
    .load()

# Write to console
query = kafka_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime="10 seconds") \
    .start()

# Keep streaming running
query.awaitTermination()

25/03/15 08:33:38 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ff0e38e5-ce97-4a92-9852-6a3fab5e8161. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/03/15 08:33:38 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/03/15 08:33:38 ERROR MicroBatchExecution: Query [id = 516f018e-3ff3-4f97-b63c-915c272d9bc5, runId = 31e39021-b67f-4922-8a36-59c4b8b3681d] terminated with error
java.lang.NoClassDefFoundError: org/apache/spark/kafka010/KafkaConfigUpdater
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$.kafkaParamsForDriver(KafkaSourceProvider.scala:645)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaScan.toMicroBatchStream(KafkaSourceProvider.scala:482)
	at org.ap

StreamingQueryException: org/apache/spark/kafka010/KafkaConfigUpdater
=== Streaming Query ===
Identifier: [id = 516f018e-3ff3-4f97-b63c-915c272d9bc5, runId = 31e39021-b67f-4922-8a36-59c4b8b3681d]
Current Committed Offsets: {}
Current Available Offsets: {}

Current State: INITIALIZING
Thread State: RUNNABLE

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Create SparkSession with explicit JAR paths
spark = SparkSession.builder \
    .appName("KafkaSparkStreaming") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar,/opt/spark/jars/kafka-clients-3.2.1.jar") \
    .config("spark.driver.extraClassPath", "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar:/opt/spark/jars/kafka-clients-3.2.1.jar") \
    .config("spark.executor.extraClassPath", "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar:/opt/spark/jars/kafka-clients-3.2.1.jar") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

# Verify Spark version (optional, for debugging)
print("Spark Version:", spark.version)

# Define Kafka DataFrame for the AIDHOOSTATION topic
kafka_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "AIDHOOSTATION") \
    .option("startingOffsets", "earliest") \
    .load()

# Cast key and value to strings for readability
processed_df = kafka_df.select(
    col("key").cast("string"),
    col("value").cast("string"),
    col("timestamp")
)

# Write to console with checkpointing
query = processed_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime="10 seconds") \
    .option("checkpointLocation", "/opt/spark/checkpoints") \
    .start()

# Keep streaming running
query.awaitTermination()

Spark Version: 3.2.1


25/03/15 08:35:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/03/15 08:35:42 ERROR MicroBatchExecution: Query [id = 38f0b42b-d256-407c-9840-7def4b63bba0, runId = 8a26f740-dc53-4a38-9c1f-7207a15c1a61] terminated with error
java.lang.NoClassDefFoundError: org/apache/spark/kafka010/KafkaConfigUpdater
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$.kafkaParamsForDriver(KafkaSourceProvider.scala:645)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaScan.toMicroBatchStream(KafkaSourceProvider.scala:482)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution$$anonfun$1.$anonfun$applyOrElse$4(MicroBatchExecution.scala:101)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution$$anonfun$1.applyOrElse(MicroBatchExecution.scala:94)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution$$anonfun$1.

StreamingQueryException: org/apache/spark/kafka010/KafkaConfigUpdater
=== Streaming Query ===
Identifier: [id = 38f0b42b-d256-407c-9840-7def4b63bba0, runId = 8a26f740-dc53-4a38-9c1f-7207a15c1a61]
Current Committed Offsets: {}
Current Available Offsets: {}

Current State: INITIALIZING
Thread State: RUNNABLE

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Create SparkSession with explicit JAR paths
spark = SparkSession.builder \
    .appName("KafkaSparkBatch") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar,/opt/spark/jars/kafka-clients-3.2.1.jar") \
    .config("spark.driver.extraClassPath", "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar:/opt/spark/jars/kafka-clients-3.2.1.jar") \
    .config("spark.executor.extraClassPath", "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar:/opt/spark/jars/kafka-clients-3.2.1.jar") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

# Verify Spark version
print("Spark Version:", spark.version)

# Define Kafka batch read for the AIDHOOSTATION topic
kafka_df = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "AIDHOOSTATION") \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load()

# Cast key and value to strings for readability
processed_df = kafka_df.select(
    col("key").cast("string").alias("key"),
    col("value").cast("string").alias("value"),
    col("timestamp"),
    col("partition"),
    col("offset")
)

# Show the results (print to console)
processed_df.show(truncate=False)


Spark Version: 3.2.1


Py4JJavaError: An error occurred while calling o316.showString.
: java.lang.NoClassDefFoundError: org/apache/spark/kafka010/KafkaConfigUpdater
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$.kafkaParamsForDriver(KafkaSourceProvider.scala:645)
	at org.apache.spark.sql.kafka010.KafkaRelation.buildScan(KafkaRelation.scala:61)
	at org.apache.spark.sql.execution.datasources.DataSourceStrategy$.apply(DataSourceStrategy.scala:339)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:63)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.plan(QueryPlanner.scala:93)
	at org.apache.spark.sql.execution.SparkStrategies.plan(SparkStrategies.scala:68)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$3(QueryPlanner.scala:78)
	at scal

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Create SparkSession with explicit JAR paths
spark = SparkSession.builder \
    .appName("KafkaSparkBatch") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar,/opt/spark/jars/kafka-clients-3.2.1.jar") \
    .config("spark.driver.extraClassPath", "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar:/opt/spark/jars/kafka-clients-3.2.1.jar") \
    .config("spark.executor.extraClassPath", "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar:/opt/spark/jars/kafka-clients-3.2.1.jar") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

# Verify Spark version
print("Spark Version:", spark.version)

# Print the classpath for debugging
print("Driver Extra Classpath:", spark.conf.get("spark.driver.extraClassPath"))
print("Executor Extra Classpath:", spark.conf.get("spark.executor.extraClassPath"))

# Define Kafka batch read for the AIDHOOSTATION topic
kafka_df = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "AIDHOOSTATION") \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load()

# Cast key and value to strings for readability
processed_df = kafka_df.select(
    col("key").cast("string").alias("key"),
    col("value").cast("string").alias("value"),
    col("timestamp"),
    col("partition"),
    col("offset")
)

# Show the results (print to console)
processed_df.show(truncate=False)

# Optionally, write to a file (e.g., CSV) for persistence
processed_df.write \
    .mode("overwrite") \
    .csv("/opt/spark/output/AIDHOOSTATION_batch_output")

# Stop the SparkSession
spark.stop()

Spark Version: 3.2.1
Driver Extra Classpath: /opt/spark/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar:/opt/spark/jars/kafka-clients-3.2.1.jar
Executor Extra Classpath: /opt/spark/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar:/opt/spark/jars/kafka-clients-3.2.1.jar


Py4JJavaError: An error occurred while calling o351.showString.
: java.lang.NoClassDefFoundError: org/apache/spark/kafka010/KafkaConfigUpdater
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$.kafkaParamsForDriver(KafkaSourceProvider.scala:645)
	at org.apache.spark.sql.kafka010.KafkaRelation.buildScan(KafkaRelation.scala:61)
	at org.apache.spark.sql.execution.datasources.DataSourceStrategy$.apply(DataSourceStrategy.scala:339)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:63)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.plan(QueryPlanner.scala:93)
	at org.apache.spark.sql.execution.SparkStrategies.plan(SparkStrategies.scala:68)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$3(QueryPlanner.scala:78)
	at scal